# 객체지향 프로그램으로 하면 좋은 점
config에 각 값 세팅해서 key값으로 불러오면 편함

In [12]:
import os
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

# 각 노드를 몇개를 설정하고, 연결하고 layer를 몇층을 쌓을 것인지
class XOR(nn.Module): # 무조건 nn.Module를 상속 받아야함.
    def __init__(self, config): # 생성자 오버라이드 해야함.
        super(XOR, self).__init__()

        # 입력층 노드 수 : 2
        self.inode = config['input_node']
        # 은닉층 데이터 크기 : 10
        self.hnode = config['hidden_node']
        # 출력층 노드 수 : 1, 분류해야하는 레이블 수
        self.onode = config['output_node']

        # 활성화 함수로 Sigmoid 사용
        self.activation = nn.Sigmoid()

        # 신경망 설계
        self.linear1 = nn.Linear(self.inode, self.hnode, bias=True) # 2*10
        self.linear2 = nn.Linear(self.hnode, self.onode, bias=True) # 10*1

    def forward(self, input_features): # 포워드도 오버라이드 해야함. # foward는 Hypothesis만드는 작업.

        output1 = self.linear1(input_features) # 인풋을 넣어서 아웃풋 하나 뽑아냄.
        hypothesis1 = self.activation(output1) # 시그모이드

        output2 = self.linear2(hypothesis1) # 그걸 다음 층에 넣음
        hypothesis2 = self.activation(output2) # 시그모이드

        return hypothesis2 # 최종 hypothesis 리턴

In [13]:
# 데이터 읽기 함수
def load_dataset(file, device):
    data = np.loadtxt(file)
    print("DATA=", data)

    input_features = data[:, 0:-1]
    print("INPUT_FEATURES=", input_features)

    labels = np.reshape(data[:, -1],(4,1))
    print("LABELS=", labels)

    input_features = torch.tensor(input_features, dtype=torch.float).to(device)
    labels = torch.tensor(labels, dtype=torch.float).to(device)

    return (input_features, labels)

In [14]:
# 모델 학습 함수
def train(config):

    # 모델 생성
    model = XOR(config) # .to()하거나 쿠다 지우면 CPU

    # 데이터 읽기
    (input_features, labels) = load_dataset(config["input_data"], "cpu")

    # 여기 중요
    # tensorDataset/DataLoader를 통해 배치 단위로 데이터를 나누고 / 셔플
    train_features = TensorDataset(input_features, labels)
    train_dataloader = DataLoader(train_features, shuffle= True, batch_size=config["batch_size"]) # 배치대로 가져오는데 셔플해서 가져옴

    # 비용함수 BCE
    loss_func = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=config["learn_rate"])

    for epoch in range(config["epoch"]+1): # 에폭만큼 수행

        # 학습 모드 셋팅(드랍아웃 같은 것 하려면 train으로. 학습이 아니라 모드 셋팅임)
        model.train()

        # epoch 마다 평균 비용을 저장하기 위한 리스트
        costs = []

        for (step, batch) in enumerate(train_dataloader): # 배치만큼 셔플해서 가져온 것
            # cpu를 통해 메모리에 업로드
            batch = tuple(t.to("cpu") for t in batch) # 배치에 하나씩 가져와서 쿠다에 업로드하는 과정

            # 각 feature 저장
            input_features, labels = batch
            
            # 역전파 변화도 초기화
            # backward() 호출 시, 변화도 버퍼에 데이터가 계속 누적한 것을 초기화
            # 파이토치는 그 전 기울기가 남아있어서 초기화 해야함.
            optimizer.zero_grad()
            
            # H(X) 계산: forward 연산
            hypothesis = model(input_features)
            # 비용 계산
            cost = loss_func(hypothesis, labels)
            # 역전파 수행
            cost.backward()
            optimizer.step()

            # 현재 batch의 스텝 별 loss 저장
            costs.append(cost.data.item())
            
        if epoch%100 == 0:
            print("Avg Loss={0:f}".format(np.mean(costs)))
            torch.save(model.state_dict(), os.path.join(config["output_dir"], "epoch_{0:d}.pt".format(epoch)))
            do_test(model, train_dataloader)

In [15]:
# 모델 평가 함수
def test(config):
    model = XOR(config)

    # 저장된 모델 가중치 로드
    model.load_state_dict(torch.load(os.path.join(config["output_dir"], config["model_name"])))

    # 데이터 load
    (features, labels) = load_dataset(config["input_data"])

    test_features = TensorDataset(features, labels)
    test_dataloader = DataLoader(test_features, shuffle=True, batch_size=config["batch_size"])

    do_test(model, test_dataloader)

def tensor2List(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

# 평가 수행 함수
def do_test(model, test_dataloader):

    # 평가모드 셋팅
    model.eval()

    # Batch 별로 예측값과 정답을 저장할 리스트 초기화
    predicts, golds = [], []
    with torch.no_grad(): # 역전파 하면 안됨.. 평가라서..
        for stop, batch in enumerate(test_dataloader):

            #.cpu()를 통해 메모리에 업로드
            batch = tuple(t.to("cpu") for t in batch)

            input_features, labels = batch
            hypothesis = model(input_features)
            logits = (hypothesis > 0.5).float() # 0, 1로 바꾸는 logit
            x = tensor2List(logits)
            y = tensor2List(labels)

            # 예측값과 정답을 리스트에 추가
            predicts.extend(x)
            golds.extend(y)

        print("PRED=", predicts)
        print("GOLD=", golds)
        print("ACC={0:f}\n".format(accuracy_score(golds, predicts)))

In [16]:
if(__name__=="__main__"):
    root_dir = '.'
    output_dir = os.path.join(root_dir, "output")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    input_data = "{0:s}/{1:s}".format(root_dir, 'train.txt')

    config = {"mode": "train", # train, test 모드 선택
              "model_name": "epoch_{0:d}.pt".format(1000), # 테스트때 사용하고 싶은 모델 고를 수 있게. 
              "output_dir": output_dir, # 저장 폴더
              "input_data": input_data, # 불러올 폴더
              "input_node": 2,
              "hidden_node": 10,
              "output_node": 1,
              "learn_rate": 1,
              "batch_size": 4,
              "epoch": 1000,
              }

    if(config["mode"] == "train"):
        train(config)
    else:
        test(config)

DATA= [[0. 0. 0.]
 [0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 0.]]
INPUT_FEATURES= [[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]]
LABELS= [[0.]
 [1.]
 [1.]
 [0.]]
Avg Loss=0.700358
PRED= [[1.0], [0.0], [1.0], [0.0]]
GOLD= [[0.0], [0.0], [1.0], [1.0]]
ACC=0.500000

Avg Loss=0.692417
PRED= [[1.0], [1.0], [0.0], [0.0]]
GOLD= [[1.0], [0.0], [1.0], [0.0]]
ACC=0.500000

Avg Loss=0.689529
PRED= [[0.0], [1.0], [1.0], [0.0]]
GOLD= [[0.0], [1.0], [0.0], [1.0]]
ACC=0.500000

Avg Loss=0.664861
PRED= [[0.0], [0.0], [0.0], [1.0]]
GOLD= [[0.0], [1.0], [0.0], [1.0]]
ACC=0.750000

Avg Loss=0.417974
PRED= [[0.0], [1.0], [0.0], [1.0]]
GOLD= [[0.0], [1.0], [0.0], [1.0]]
ACC=1.000000

Avg Loss=0.113380
PRED= [[1.0], [0.0], [0.0], [1.0]]
GOLD= [[1.0], [0.0], [0.0], [1.0]]
ACC=1.000000

Avg Loss=0.049985
PRED= [[1.0], [0.0], [0.0], [1.0]]
GOLD= [[1.0], [0.0], [0.0], [1.0]]
ACC=1.000000

Avg Loss=0.030372
PRED= [[1.0], [0.0], [1.0], [0.0]]
GOLD= [[1.0], [0.0], [1.0], [0.0]]
ACC=1.000000

Avg Loss=0.021414
PRED= [[0.0], [1.0], [0.0